In [1]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as preproc 
import pandas as pd
import numpy as np
import preprocessing

In [2]:
GSPREADHSEET_DOWNLOAD_URL = (
    "https://docs.google.com/spreadsheets/d/{gid}/export?format=csv&id={gid}".format
)

FIUFIP_2021_1_GID = '1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0'
df = pd.read_csv(GSPREADHSEET_DOWNLOAD_URL(gid=FIUFIP_2021_1_GID))

In [3]:
# Preprocesamiento

# se quitan features no utiles

df = preprocessing.remove_irrelevant_features(df)
# df.drop('educacion_alcanzada', axis='columns', inplace = True)
# df.drop('barrio', axis='columns', inplace = True)

df = preprocessing.missings_treatment(df)
# df['categoria_de_trabajo'] = df['categoria_de_trabajo'].replace(np.nan, 'sin_informar')
# df['trabajo'] = df['trabajo'].replace(np.nan, 'sin_informar')
# df.loc[df['categoria_de_trabajo'] == 'sin_trabajo', ['trabajo']] = 'sin_trabajo'

In [4]:
# se aplica one hot encoding (menos 1) para convertir las variables categoricas

df = preprocessing.one_hot_encodding(df)

In [5]:
# separo en set de entrenamiento y set de validacion usando la biblioteca model_selection de sklearn
X_train, X_val, y_train, y_val = preprocessing.dataset_split(df)

X_train

,anios_estudiados,edad,ganancia_perdida_declarada_bolsa_argentina,horas_trabajo_registradas,categoria_de_trabajo_empleado_municipal,categoria_de_trabajo_empleado_provincial,categoria_de_trabajo_monotibutista,categoria_de_trabajo_relacion_de_dependencia,categoria_de_trabajo_responsable_inscripto,categoria_de_trabajo_sin_informar,...,trabajo_profesional_especializado,trabajo_reparador,trabajo_sector_primario,trabajo_seguridad,trabajo_servicio_domestico,trabajo_sin_informar,trabajo_sin_trabajo,trabajo_soporte_tecnico,trabajo_transporte,trabajo_ventas
32098,13,45,7298,60,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
25206,13,47,1831,38,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23491,14,48,0,50,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
12367,13,29,0,40,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
7054,8,23,0,25,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13123,18,43,0,40,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19648,10,44,0,35,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
9845,14,23,0,40,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
10799,20,45,7688,45,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# ejemplo knn (1er ejemplo - le pase los valores numericos sin modificar)

# importo la biblioteca
from sklearn.neighbors import KNeighborsClassifier

# construyo el modelo
knn = KNeighborsClassifier(n_neighbors=20, weights= 'uniform')  # se usa un numero especifico de vecinos (20), pero esto es un HIPERPARAMETRO 
knn.fit(X_train, y_train)


# lo uso para predecir
y_pred = knn.predict(X_val)

# comparo los resultados
print(f"{np.sum(y_val == y_pred)} predicciones correctas de {len(y_val)}")

8303 predicciones correctas de 9769


In [7]:
# normalizo usando preprocessing de sklearn

normalizer = preprocessing.Normalizer()
X_train_n = normalizer.fit(X_train).transform(X_train)
X_val_n = normalizer.fit(X_val).transform(X_val)
X_train_n[0]

AttributeError: module 'preprocessing' has no attribute 'Normalizer'

In [ ]:
# pruebo usando los valores normalizados, pero da ligeramente peor, da mejor sin eso, al menos con 20 vecinos.

# construyo el modelo
knn = KNeighborsClassifier(n_neighbors=20, weights= 'uniform')  # se usa un numero especifico de vecinos (20), pero esto es un HIPERPARAMETRO 
knn.fit(X_train_n, y_train)


# lo uso para predecir
y_pred = knn.predict(X_val_n)

# comparo los resultados
print(f"{np.sum(y_val == y_pred)} predicciones correctas de {len(y_val)}")

In [ ]:
# pruebo usando la funcion distancia, tambien da peor, al menos con 20 vecinos.

knn = KNeighborsClassifier(n_neighbors=20, weights= 'distance')  # se usa un numero especifico de vecinos (20), pero esto es un HIPERPARAMETRO 
knn.fit(X_train, y_train)


# lo uso para predecir
y_pred = knn.predict(X_val)

# comparo los resultados
print(f"{np.sum(y_val == y_pred)} predicciones correctas de {len(y_val)}")

In [ ]:
# falta generalizar las busquedas de la mejor combinacion de features + hiperparametros y ver graficos